
# 🤖 Aula 4: Criando um Chatbot com Memória

Neste notebook, vamos construir um chatbot que lembra informações do usuário e melhora suas respostas ao longo do tempo.

## 📌 O que faremos:
1. **Entender como funciona a memória em AI Agents**: curto e longo prazo.
2. **Fundamentação matemática**: Como vetores semânticos e embeddings são usados para recuperar memória.
3. **Implementar um chatbot aprimorado**: Usando FAISS para armazenar memórias e melhorar retenção.
4. **Comparar diferentes estratégias de memória**.
5. **Completar um desafio prático!**

---

## 📖 Como um Chatbot Lembra de Informações?

A maioria dos chatbots convencionais não tem uma "memória real" – eles simplesmente processam a conversa atual e esquecem interações passadas. No entanto, podemos usar diferentes estratégias para adicionar memória:

### Tipos de Memória em AI Agents:

1. **Memória de Curto Prazo (Contextual)**: Mantém as últimas interações no histórico da conversa.
2. **Memória de Longo Prazo (Persistente)**: Salva informações importantes para uso posterior.
3. **Memória Baseada em Embeddings**: Usa vetores semânticos para recuperar informações relevantes.

### Matemática por Trás da Memória em AI Agents

A memória pode ser modelada matematicamente usando **representação vetorial**:

- Cada mensagem \( M \) pode ser convertida em um vetor \( v \) usando **embeddings**.
- Um banco de dados vetorial, como **FAISS**, pode armazenar esses vetores para permitir buscas eficientes.
- Quando uma nova pergunta é feita, encontramos os vetores mais próximos para recuperar contexto relevante.



In [ ]:

# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai


In [ ]:

import openai

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um histórico de mensagens para armazenar contexto
historico = [{"role": "system", "content": "Você é um assistente amigável."}]

def chatbot_memoria(pergunta):
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste rápido
print(chatbot_memoria("Meu nome é João."))
print(chatbot_memoria("Qual é o meu nome?"))


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai faiss-cpu numpy

import openai
import numpy as np
import faiss

# Defina sua chave de API aqui (substitua 'SUA_CHAVE_AQUI' pela sua chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Criar banco de memória vetorial com FAISS
dimensao = 1536  # Tamanho dos embeddings da OpenAI
indice_memoria = faiss.IndexFlatL2(dimensao)  # Banco de dados vetorial
memorias = []  # Lista de strings armazenadas

# Função para obter embedding de um texto
def obter_embedding(texto):
    resposta = openai.Embedding.create(input=texto, model="text-embedding-ada-002")
    return np.array(resposta["data"][0]["embedding"], dtype=np.float32)

# Função para armazenar memória
def armazenar_memoria(texto):
    vetor = obter_embedding(texto)
    memorias.append(texto)
    indice_memoria.add(np.array([vetor]))

# Função para recuperar memória mais próxima
def recuperar_memoria(texto):
    if len(memorias) == 0:
        return None
    
    vetor = obter_embedding(texto)
    D, I = indice_memoria.search(np.array([vetor]), k=1)
    return memorias[I[0][0]] if D[0][0] < 0.5 else None  # Se a distância for pequena, retornamos

# Chatbot com memória aprimorada
historico = [{"role": "system", "content": "Você é um assistente amigável."}]

def chatbot_com_memoria(pergunta):
    memoria_relevante = recuperar_memoria(pergunta)
    if memoria_relevante:
        pergunta = f"Com base na memória: '{memoria_relevante}', {pergunta}"
    
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    
    armazenar_memoria(pergunta)  # Armazenar nova memória
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste aprimorado
print(chatbot_com_memoria("Meu nome é Ana."))
print(chatbot_com_memoria("O que eu te disse antes?"))




## 🎯 Desafio para os alunos!
1. Modifique o chatbot para lembrar mais detalhes da conversa.
2. Teste limitar a memória do chatbot para evitar respostas irrelevantes.
3. Compartilhe suas descobertas no fórum do curso!

---
